In [10]:
import gym
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from IPython.display import clear_output

In [2]:

#you need to pip install Box2d for 
#THE BOX2D ENVS DON'T INSTALL CORRECTLY UNLESS YOU HAVE SWIG INSTALLED
#ONE YOU INSTALL SWIG YOU NEED TO REINSTALL GYM
#AFTER SWIG YOU NEED TO INSTALL MICROSOFT VISUAL STUDIO BUILD TOOLS WITH THE SDK(THE SDK IS OPTIONAL AT INSTALL BUT YOU NEED IT)
try:
    env = gym.make("CarRacing-v0")
except AttributeError:
    print("(this is a custom error message written by me) Attribute Error: try installing box2d with 'pip install Box2d'. If you get an error doing that open this file and read the comment above this try/except block")
    exit(1)
discount_rate=0.97
learning_rate=0.001

c:\users\eric\projects\rl experiments\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
print(env.action_space)

Box(-1.0, 1.0, (3,), float32)


In [4]:
class CustomSquaredError(keras.losses.Loss):#just works for one prediction
  def call(self, y_true, y_pred):
    #print(f"smd 2, true: {type(y_true)},pred:{type(y_pred)}")
    res = tf.math.squared_difference(tf.reduce_max(y_pred),y_true)
    #print("smd 2.5")
    return res

In [5]:
#kernel size article: https://medium.com/analytics-vidhya/how-to-choose-the-size-of-the-convolution-filter-or-kernel-size-for-cnn-86a55a1e2d15
model = keras.Sequential(layers=[
    keras.layers.Conv2D(filters=1,kernel_size=(3,3),activation="relu",input_shape=(96,96,3)),
    keras.layers.MaxPool2D(),#we are keeping pool size at the default 2x2 and strides=None
    keras.layers.Flatten(),
    #https://www.alexirpan.com/public/research/281areport.pdf
    #this is an example of an additive q-function
    #the output format is this: 
    #    [steer left, no steer, steer right, accelerate, coast, brake]
    #where each value in the shape is the q-value for that action
    keras.layers.Dense(6)
])
model.compile(loss=CustomSquaredError())
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 94, 94, 1)         28        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 47, 47, 1)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2209)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 13260     
Total params: 13,288
Trainable params: 13,288
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
def train(state,action,next_state,reward):
    next_pred = model.predict(tf.expand_dims(next_state,0))
    q_next = next_pred
    q_update = reward+discount_rate*q_next
    model.train_on_batch(x=tf.expand_dims(state,0),y=q_update)

def makeDecision(state):
    action_greedy = np.argmax(model.predict(tf.expand_dims(state,0)))
    action_random = random.randint(0,3)
    return action_greedy if random.random()>epsilon else action_random

In [12]:
episodes = 1
for episode in range(episodes):
    done=False
    state = env.reset()
    iteration=0
    while not done:
        action = model.predict(tf.expand_dims(state,0))[0]
        print(action)
        steerAction = np.argmax(action[0:3])
        pedalAction = np.argmax(action[3:6])
        usableAction = [steerAction-1,1 if pedalAction==0 else 0, 1 if pedalAction==2 else 0]
        #action should be a 3-tuple like this: [steer, gas, brake]
        next_state, reward, done, info = env.step(usableAction)#reward is one if the goal is reached and zero if the goal isn't reached
        #train(state,action,next_state,reward)
        state=next_state
        #train((state,action,next_state,reward,done))
        #print(reward)
        env.render()
        clear_output(wait=True)
print("done!")


[ -48.932625 -164.10951  -155.2628     68.47785   380.82953   -79.54268 ]
